<a href="https://colab.research.google.com/github/OmarMeriwani/CE807-Sentiment-analysis/blob/master/Frequent_Subjective_Words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This file contains the code of one of the features that has been used in features extraction file. The method is based on selecting words with specific POS tags, and storing them with the value assigned to them in training file. In addition to the number of occurances in the whole corpus. The NER words were removed from the final sets.   

In [0]:
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize
from stanfordcorenlp import StanfordCoreNLP
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import RegexpTokenizer
import os
from senticnet.senticnet import SenticNet
from nltk.stem.porter import *

java_path = "C:/Program Files/Java/jdk1.8.0_161/bin/java.exe"
os.environ['JAVAHOME'] = java_path
host='http://localhost'
port=9000
scnlp =StanfordCoreNLP(host, port=port,lang='en', timeout=30000)
stemmer = PorterStemmer()
stop_words = stopwords.words('english')
df = pd.read_csv('train.csv',header=0,sep='\t')
prev = ''
tknzr = RegexpTokenizer(r'\w+')
ListOfCleanTokens = []



Only full sentences were used in this feature extraction step, in the loop below, the steps are going to be done on word level, inside each sentence.

In [0]:
for i in range(0,len(df)):
    if prev != str(df.loc[i][1]):
        sentence = df.loc[i][2]
        prev = str(df.loc[i][1])
    else:
        continue
    #^^ Above lines are to choose only full sentences from the training set.
    
    '''
    Tokenize, Get NER and POS tags
    '''
    sentences = sent_tokenize(sentence)
    reviewPolarity = int(df.loc[i][3])
    tokens = []
    for sent in sentences:
        t = tknzr.tokenize(sent)
        for tk in t:
            tokens.append(tk)
    NER = scnlp.ner(sentence)
    POStagged = scnlp.pos_tag(sentence)
    
    '''
    Remove stop words, convert to lower case and choose only unique set of words from the sentence
    '''
    cleanTokensWithoutNegation = [str(t).lower() for t in tokens if t not in stop_words]
    cleanTokensWithoutNegation = set(cleanTokensWithoutNegation)
    '''
    Remove NER and choose specific POS Tags, then only words that are not NER and within the set of tags will be chosen and added to the dataset.
    '''
    usefultags = ['JJ','JJR','JJS','RB ','VB ','VBD','VBG','VBN','VBP','VBZ']
    cleanTokensWithoutNegation = [t for t in cleanTokensWithoutNegation
                                  if t not in
                                  [str(n).lower() for n, nrt in NER if str(nrt).lower() != 'o'] and
                                  t in [str(word).lower() for word, p in POStagged if p in usefultags ]]
    ListOfCleanTokens.append([cleanTokensWithoutNegation, reviewPolarity])


After preparing the set of words with their polarity, and that are within the POS tag set and are not NER, the count of each word will be set, and according to the polarity, the set is going to be divided into positive words (the words that comes only within the positive reviews) and negative words.

In [0]:
GoodWords = {}
BadWords = {}
for i in ListOfCleanTokens:
    cleanTokens = i[0]
    reviewPolarity = i[1]
    if reviewPolarity > 2:
        for t in cleanTokens:
            if t not in GoodWords:
                GoodWords[t] = 1
            else:
                GoodWords[t] = GoodWords.get(t) + 1
    if reviewPolarity < 2:
        for t in cleanTokens:
            if t not in BadWords:
                BadWords[t] = 1
            else:
                BadWords[t] = BadWords.get(t) + 1


Remove the common words between the two lists. and store the lists.

In [0]:




intersection = [t for t,i in GoodWords.items() if t in BadWords]
for i in intersection:
    if i in GoodWords:
        GoodWords.pop(i)
for i in intersection:
    if i in BadWords:
        BadWords.pop(i)

print(GoodWords)
print(BadWords)
import csv
with open('PositiveWords.csv', 'w') as f:
    for key in GoodWords.keys():
        f.write("%s,%s\n"%(key,GoodWords[key]))
with open('NegativeWords.csv', 'w') as f:
    for key in BadWords.keys():
        f.write("%s,%s\n"%(key,BadWords[key]))
